#### Creating five tabular data for databases:
1. Stock Market Index (Country, Index Name, Index Ticker)
2. Stock Market Index Composition (Index Shortname, Company Name, Company Ticker, Industry, Market Cap, Current Price, Avg. Annual Return)
3. Company (Company Ticker, Company’s Name, Company’s Address, Market Cap, )
4. Company Financial Statements (Company ticker, company’s name, year, Income Statement, Balance Sheets, Cash Flow)
5. Company Daily Stock Price (Last 1 year)

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf

#### 1. Stock Market Index (Country, Index Name, Index Shortname, Index Ticker)

In [ ]:
# Define the stock market index by country
smi = {
    "Country" : ["France","Germany","Spain"],
    "Index_Name" : ["Cotation Assistee en Continu","Deutscher Aktienindex","Indice Bursatil Espanol"],
    "Index_Shortname" : ["CAC 40","DAX 40","IBEX 35"],
    "Index_Ticker" : ["FCHI","GDAXI","IBEX"]
}
stock_market_index = pd.DataFrame(smi)

stock_market_index.to_csv('../data/stock_market_index.csv', index=False)

In [38]:
stock_market_index

,Country,Index_Name,Index_Shortname,Index_Ticker
0,France,Cotation Assistee en Continu,CAC 40,FCHI
1,Germany,Deutscher Aktienindex,DAX 40,GDAXI
2,Spain,Indice Bursatil Espanol,IBEX 35,IBEX


#### 2. Stock Market Index Composition (Index Shortname, Company Name, Company Ticker, Industry, Market Cap, Current Price, Avg. Annual Return)

In [ ]:
# Define stock market index by country
indices = {
        "France" : ["CAC 40","^FCHI",
                    ["AC.PA","ACA.PA","AI.PA","AIR.PA","BN.PA","BNP.PA","CA.PA","CAP.PA","CS.PA",
                     "DG.PA","DSY.PA","EDEN.PA","EL.PA","EN.PA","ENGI.PA","ERF.PA","GLE.PA","HO.PA",
                     "KER.PA","LR.PA","MC.PA","ML.PA","MT.PA","OR.PA","ORA.PA","PUB.PA","RI.PA",
                     "RMS.PA","RNO.PA","SAF.PA","SAN.PA","SGO.PA","STLAP.PA","STMPA.PA","SU.PA",
                     "TEP.PA","TTE.PA","URW.PA","VIE.PA","VIV.PA"]],
        "Germany" : ["DAX 40","^GDAXI",
                     ['ADS.DE','AIR.DE','ALV.DE', 'BAS.DE', 'BAYN.DE', 'BEI.DE', 'BMW.DE', 'CON.DE',
                      '1COV.DE', 'DHER.DE', 'DTE.DE', 'DPW.DE', 'DB1.DE', 'DBK.DE', 'ENR.DE', 'FRE.DE',   
                      'FME.DE', 'HEN3.DE', 'HNR1.DE', 'IFX.DE', 'LIN.DE', 'MRK.DE', 'MTX.DE', 'MUV2.DE',
                      'PUM.DE', 'RWE.DE', 'SAP.DE', 'SHL.DE', 'SIE.DE', 'VNA.DE', 'VOW3.DE', 'ZAL.DE',
                      'HEI.DE', 'BAYN.DE', 'SY1.DE', 'HFG.DE', 'MBG.DE', 'HLE.DE', 'BNR.DE', 'TYO.DE']],
        "Spain" : ["IBEX 35","^IBEX",
                   ["ACS.MC","ACX.MC","AENA.MC","AMS.MC","ANA.MC","ANE.MC","BBVA.MC","BKT.MC","CABK.MC",
                    "CLNX.MC","COL.MC","ELE.MC","ENG.MC","FDR.MC","FER.MC","GRF.MC","IAG.MC","IBE.MC",
                    "IDR.MC","ITX.MC","LOG.MC","MAP.MC","MRL.MC","MTS.MC","NTGY.MC","PUIG.MC","RED.MC",
                    "REP.MC","ROVI.MC","SAB.MC","SAN.MC","SCYR.MC","SLR.MC","TEF.MC","UNI.MC"]]
            }

index_composition = {}

In [ ]:
for country in list(indices.keys()): 
    # Stock Index and Industry
    index = yf.Ticker(indices[country][1])
    index_data = index.history(period="5y")

    index_tickers = indices[country][2]

    # Create an empty list to hold company information
    company_data = []

    # Loop through each ticker and fetch the required data
    for ticker in index_tickers:
        stock = yf.Ticker(ticker)

        # Get the stock info for each company
        stock_info = stock.info

        # Extract relevant information
        company_name = stock_info.get('longName', 'N/A')  # Company Name
        industry = stock_info.get('industry', 'N/A')      # Industry
        market_cap = stock_info.get('marketCap', '')   # Market Capitalization
        def calculate_average_annual_return(ticker):
            # Download the stock data for the past 5 years (adjustable)
            stock_data = yf.download(ticker, period="5y", interval="1d")

            # Ensure we're using the 'Adj Close' prices (adjusted for splits/dividends)
            stock_data['Return'] = stock_data['Adj Close'].pct_change()

            # Resample to get yearly returns, assuming business year frequency
            yearly_returns = stock_data['Adj Close'].resample('Y').ffill().pct_change()

            # Drop NaN for the first year
            yearly_returns = yearly_returns.dropna()

            # Calculate the average annual return
            avg_annual_return = yearly_returns.mean()

            # Format it as percentage
            avg_annual_return_percentage = round(avg_annual_return * 100,2)

            return avg_annual_return_percentage

        try:
            # Download stock data for the last 5 days
            stock_data = yf.download(ticker, period="5y", interval="1d")

            # Get the latest adjusted close price if available
            if not stock_data.empty:
                close_price = round(stock_data['Adj Close'][-1],2)
                average_return = calculate_average_annual_return(ticker)
            else:
                close_price = ''
                average_return = ''

        finally:
            # Append the data to the list
            company_data.append([ticker, company_name, industry, market_cap, close_price, average_return])

    # Create a DataFrame from the list
    df = pd.DataFrame(company_data, columns=['Ticker', 'Company_Name', 'Industry', 'Market_Cap', 'Current_Price','Avg_Annual_Return'])
    df["Market_Cap"] = pd.to_numeric(df["Market_Cap"])
    df['Avg_Annual_Return'] = pd.to_numeric(df['Avg_Annual_Return'])
    df['Current_Price'] = pd.to_numeric(df['Current_Price'])
    df = df.sort_values(by="Ticker", ascending=True)
    df["Index_Shortname"]=indices[country][0]
    df = df[['Index_Shortname','Ticker', 'Company_Name', 'Industry', 'Market_Cap', 'Current_Price','Avg_Annual_Return']]
    index_composition[country] = df.reset_index(drop=True)
    
index_compositions = pd.concat([index_composition["France"],index_composition["Germany"]])
index_compositions = pd.concat([index_compositions,index_composition["Spain"]])
index_compositions['Company_Name'] = index_compositions['Company_Name'].str.replace('é', 'e')
index_compositions['Company_Name'] = index_compositions['Company_Name'].str.replace('è', 'e')
index_compositions = index_compositions.applymap(lambda x: x.encode('ascii', 'ignore').decode('ascii') if isinstance(x, str) else x)


index_compositions.to_csv('../data/index_compositions.csv', index=False)

In [41]:
index_compositions

,Index_Shortname,Ticker,Company_Name,Industry,Market_Cap,Current_Price,Avg_Annual_Return
0,CAC 40,AC.PA,Accor SA,Lodging,9.856987e+09,40.46,4.21
1,CAC 40,ACA.PA,Credit Agricole S.A.,Banks - Regional,4.190949e+10,13.81,10.80
2,CAC 40,AI.PA,L'Air Liquide S.A.,Specialty Chemicals,9.850420e+10,170.78,13.20
3,CAC 40,AIR.PA,Airbus SE,Aerospace & Defense,1.049240e+11,132.92,3.61
4,CAC 40,BN.PA,Danone S.A.,Packaged Foods,4.181815e+10,64.92,2.34
...,...,...,...,...,...,...,...
30,IBEX 35,SAN.MC,"Banco Santander, S.A.",Banks - Diversified,7.031408e+10,4.57,10.56
31,IBEX 35,SCYR.MC,"Sacyr, S.A.",Engineering & Construction,2.479955e+09,3.27,11.37
32,IBEX 35,SLR.MC,"Solaria Energa y Medio Ambiente, S.A.",Utilities - Renewable,1.474422e+09,11.80,38.44
33,IBEX 35,TEF.MC,"Telefnica, S.A.",Telecom Services,2.481319e+10,4.40,4.70


#### 3. Company (Company Ticker, Company’s Name, Company’s Address, Market Cap)

In [43]:
company_details = []

In [44]:
for country in list(indices.keys()):
    for ticker in indices[country][2]:
        company = yf.Ticker(ticker)
        company_info = company.info
        index = indices[country][0]
        ticker = ticker
        longname = company_info.get('longName', 'N/A')
        industry = company_info.get('industry', 'N/A')
        sector = company_info.get('sector', 'N/A')
        address = f"{company_info.get('address1', 'N/A')}, {company_info.get('city', 'N/A')}"
        num_employees = company_info.get('fullTimeEmployees', 'N/A')
        website = company_info.get('website', 'N/A')
        
        company_details.append([index, ticker, longname, industry, sector, address, num_employees, website])

df = pd.DataFrame(company_details, columns=['Index_Shortname', 'Ticker', 'Company_Name', 'Industry', 'Sector', 'Address','Num_Employees','Website'])
df['Company_Name'] = df['Company_Name'].str.replace('é', 'e')
df = df.applymap(lambda x: x.encode('ascii', 'ignore').decode('ascii') if isinstance(x, str) else x)

df.to_csv('../data/company_details.csv', index=False)

404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/DPW.DE?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=DPW.DE&crumb=mF7sYyVSsuM
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/TYO.DE?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=TYO.DE&crumb=mF7sYyVSsuM
/var/folders/yq/9f_qv9bn6s96tkmkj2b3tsmm0000gn/T/ipykernel_25356/1887524409.py:18: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.encode('ascii', 'ignore').decode('ascii') if isinstance(x, str) else x)


In [42]:
df

,Index_Shortname,Ticker,Company_Name,Industry,Market_Cap,Current_Price,Avg_Annual_Return
0,IBEX 35,ACS.MC,"ACS, Actividades de Construcción y Servicios, ...",Engineering & Construction,1.048441e+10,42.52,14.59
1,IBEX 35,ACX.MC,"Acerinox, S.A.",Steel,2.270476e+09,9.11,5.47
2,IBEX 35,AENA.MC,"Aena S.M.E., S.A.",Airports & Air Services,2.983500e+10,198.90,7.30
3,IBEX 35,AMS.MC,"Amadeus IT Group, S.A.",Information Technology Services,2.864841e+10,65.76,0.50
4,IBEX 35,ANA.MC,"Acciona, S.A.",Engineering & Construction,6.505688e+09,119.50,10.83
5,IBEX 35,ANE.MC,"Corporación Acciona Energías Renovables, S.A.",Utilities - Renewable,6.541125e+09,20.18,-11.72
6,IBEX 35,BBVA.MC,"Banco Bilbao Vizcaya Argentaria, S.A.",Banks - Diversified,5.352944e+10,9.29,22.30
7,IBEX 35,BKT.MC,"Bankinter, S.A.",Banks - Regional,6.770399e+09,7.53,19.90
8,IBEX 35,CABK.MC,"CaixaBank, S.A.",Banks - Regional,3.865795e+10,5.36,23.71
9,IBEX 35,CLNX.MC,"Cellnex Telecom, S.A.",Real Estate Services,2.495509e+10,35.37,5.06


#### 4. Company Financial Statements (Company ticker, company’s name, year, Income Statement, Balance Sheets, Cash Flow)

In [ ]:
financial_statements = {}

In [ ]:
for country in list(indices.keys()):
    for ticker in indices[country][2]:
        if ticker not in ["DPW.DE","TYO.DE","PUIG.MC"]:
            company = yf.Ticker(ticker)

            # Income Statement    
            income_stmt = company.income_stmt.reset_index()
            income_stmt = income_stmt.iloc[:, :5]
            income_stmt.columns = ['Income Statement','2023','2022','2021','2020']
            list_income_stmt = ['Total Revenue','Cost of Revenue','Gross Profit','Operating Income','Net Income','Diluted EPS']
            income_stmt = income_stmt[income_stmt['Income Statement'].isin(list_income_stmt)]
            income_stmt[['2023','2022','2021','2020']] = income_stmt[['2023','2022','2021','2020']].astype(float) 
            income_stmt['Income Statement'] = pd.Categorical(income_stmt['Income Statement'], categories=list_income_stmt, ordered=True)
            income_stmt = income_stmt.sort_values('Income Statement')
            income_stmt.reset_index(drop=True, inplace=True)
            income_stmt = pd.melt(income_stmt, id_vars=['Income Statement'], var_name='Year', value_name='Value')
            income_stmt["Financial_Statements"]="Income Statement"
            income_stmt.columns = ['Details', 'Year', 'Value', 'Financial_Statements']
            income_stmt=income_stmt[["Financial_Statements",'Details', 'Year', 'Value']]

            # Balance Sheet
            balance_sheet = company.balance_sheet.reset_index()
            balance_sheet = balance_sheet.iloc[:, :5]
            balance_sheet.columns = ['Balance Sheet','2023','2022','2021','2020']
            list_balance_sheet = ['Current Assets','Total Assets','Current Liabilities','Long Term Debt And Capital Lease Obligation','Stockholders Equity']
            balance_sheet = balance_sheet[balance_sheet['Balance Sheet'].isin(list_balance_sheet)]
            balance_sheet[['2023','2022','2021','2020']] = balance_sheet[['2023','2022','2021','2020']].astype(float) 
            balance_sheet['Balance Sheet'] = pd.Categorical(balance_sheet['Balance Sheet'], categories=list_balance_sheet, ordered=True)
            balance_sheet = balance_sheet.sort_values('Balance Sheet')
            balance_sheet.reset_index(drop=True, inplace=True)
            balance_sheet['Balance Sheet'] = balance_sheet['Balance Sheet'].replace('Long Term Debt And Capital Lease Obligation', 'Long Term Debt')
            balance_sheet = pd.melt(balance_sheet, id_vars=['Balance Sheet'], var_name='Year', value_name='Value')
            balance_sheet["Financial_Statements"]="Balance Sheet"
            balance_sheet.columns = ['Details', 'Year', 'Value', 'Financial_Statements']
            balance_sheet=balance_sheet[["Financial_Statements",'Details', 'Year', 'Value']]

            # Cash Flow
            cash_flow = company.cashflow.reset_index()
            cash_flow = cash_flow.iloc[:, :5]
            cash_flow.columns = ['Cash Flow','2023','2022','2021','2020']
            list_cash_flow = ['Operating Cash Flow','Investing Cash Flow','Financing Cash Flow','Free Cash Flow']
            cash_flow = cash_flow[cash_flow['Cash Flow'].isin(list_cash_flow)]
            cash_flow[['2023','2022','2021','2020']] = cash_flow[['2023','2022','2021','2020']].astype(float) 
            cash_flow['Cash Flow'] = pd.Categorical(cash_flow['Cash Flow'], categories=list_cash_flow, ordered=True)
            cash_flow = cash_flow.sort_values('Cash Flow')
            cash_flow.reset_index(drop=True, inplace=True)
            cash_flow = pd.melt(cash_flow, id_vars=['Cash Flow'], var_name='Year', value_name='Value')
            cash_flow["Financial_Statements"]="Cash Flow"
            cash_flow.columns = ['Details', 'Year', 'Value', 'Financial_Statements']
            cash_flow=cash_flow[["Financial_Statements",'Details', 'Year', 'Value']]

            fin_stat = pd.concat([income_stmt, balance_sheet])
            fin_stat = pd.concat([fin_stat, cash_flow])
            fin_stat["Ticker"] = ticker
            fin_stat = fin_stat[["Ticker","Financial_Statements",'Details', 'Year', 'Value']]
            
            financial_statements[ticker] = fin_stat.reset_index(drop=True)

fin_stat = financial_statements["AC.PA"]        
    
for country in list(indices.keys()):
    for ticker in indices[country][2]:
        if ticker not in ["AC.PA","DPW.DE","TYO.DE","PUIG.MC"]:
            fin_stat = pd.concat([fin_stat,financial_statements[ticker]])

fin_stat.to_csv('../data/financial_statements.csv', index=False)

In [47]:
fin_stat

,Ticker,Financial_Statements,Details,Year,Value
0,AC.PA,Income Statement,Total Revenue,2023,5.056000e+09
1,AC.PA,Income Statement,Gross Profit,2023,2.554000e+09
2,AC.PA,Income Statement,Operating Income,2023,7.240000e+08
3,AC.PA,Income Statement,Net Income,2023,6.330000e+08
4,AC.PA,Income Statement,Diluted EPS,2023,2.220000e+00
...,...,...,...,...,...
35,UNI.MC,Cash Flow,Free Cash Flow,2021,1.392707e+10
36,UNI.MC,Cash Flow,Operating Cash Flow,2020,2.027910e+09
37,UNI.MC,Cash Flow,Investing Cash Flow,2020,1.031680e+08
38,UNI.MC,Cash Flow,Financing Cash Flow,2020,-2.270400e+07


#### 5. Company Daily Stock Price (Last 1 year)

In [ ]:
stock_price = {}

In [ ]:
for country in list(indices.keys()):
    for ticker in indices[country][2]:
        data = yf.download(ticker, period='1y')
        data.reset_index(inplace=True)
        data["Ticker"] = ticker
        data = data[["Ticker",'Date','Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']]
        data.columns = ["Ticker",'Date','Open', 'High', 'Low', 'Close', 'Adj_Close', 'Volume']
        
        stock_price[ticker] = data.reset_index(drop=True)
        
daily_stock_price = stock_price["AC.PA"]        
    
for country in list(indices.keys()):
    for ticker in indices[country][2]:
        if ticker not in ["AC.PA"]:
            daily_stock_price = pd.concat([daily_stock_price,stock_price[ticker]])

daily_stock_price.to_csv('../data/daily_stock_price.csv', index=False)

In [ ]:
data = yf.download(ticker, period='1y')
data.reset_index(inplace=True)
data["Ticker"] = ticker

In [48]:
data

,Ticker,Date,Open,High,Low,Close,Adj_Close,Volume
0,UNI.MC,2023-10-11,1.021,1.038,1.021,1.035,0.989335,4890668
1,UNI.MC,2023-10-12,1.040,1.043,1.037,1.040,0.994115,2929597
2,UNI.MC,2023-10-13,1.036,1.039,1.001,1.007,0.962571,9499582
3,UNI.MC,2023-10-16,1.004,1.019,1.004,1.017,0.972130,3419076
4,UNI.MC,2023-10-17,1.015,1.022,1.009,1.015,0.970218,3884828
...,...,...,...,...,...,...,...,...
252,UNI.MC,2024-10-07,1.080,1.087,1.073,1.082,1.082000,5600614
253,UNI.MC,2024-10-08,1.075,1.087,1.065,1.081,1.081000,5032498
254,UNI.MC,2024-10-09,1.079,1.080,1.070,1.075,1.075000,4409468
255,UNI.MC,2024-10-10,1.069,1.079,1.058,1.065,1.065000,5531232
